In [18]:
import pandas as pd
from SQLCode import DatabaseConnection
from SQLCode import DatabaseCredentials as DBC
import numpy as np

In [19]:
# Opening connection
creds = DBC.DataBaseCredentials()
conn = DatabaseConnection.sql_connection(creds.server, creds.database, creds.user, creds.password)
connection = conn.open()
cursor = connection.cursor()

In [20]:
# Getting the live_feed data
# liveFeed = pd.read_sql_query("select * from live_feed where gameID >= 20102011", connection)
# liveFeed = pd.read_sql('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/live_feed.csv')
liveFeed = pd.read_sql_query("select * from live_feed where gameID = 2020030312", connection)

In [21]:
# Getting the seasons data
seasons = pd.read_sql_query("select * from schedules", connection)

In [22]:
# boxscores = pd.read_sql_query("select gameID, teamID, playerID from box_scores where gameID >= 2010010001", connection)
boxscores = pd.read_sql_query("select gameID, teamID, playerID from box_scores where gameID =2020030312", connection)

In [23]:
# Filtering to regular seasons games and 20102011 onwards (when live data started)
seasonsFiltered = seasons[seasons['seasonID'] >= 20102011]
seasonsFiltered = seasonsFiltered[seasonsFiltered['gameType'] == 'R']

In [24]:
# Getting and filtering the raw data
rawData = pd.merge(liveFeed,seasons, how='left',on='gameID')
rawData = rawData[rawData['seasonID'] >= 20102011]
# rawData = rawData[rawData['gameType'] == 'R']
rawData = rawData[rawData['eventSubID'] == 0]
rawData = rawData[['eventID',
         'gameID',
         'eventTypeID',
         'eventDescription',
         'periodNum',
         'periodTime',
         'xCoordinate',
         'yCoordinate',
         'teamID','homeTeamID','awayTeamID','playerID','penaltyMinutes']]

In [25]:
# Merging box score data
rawData = pd.merge(rawData, 
                   boxscores, 
                   how='left', 
                   left_on=['gameID', 'playerID'], 
                   right_on=['gameID', 'playerID'],
                   suffixes=('', '_box'))

In [26]:
# Filtering to test game
rawData = rawData[rawData['gameID'] == 2020030312]

In [27]:
# Getting only the desired events
rawDataFiltered = rawData[rawData['eventTypeID'].isin(['FACEOFF',
                             'SHOT',
                             'MISSED_SHOT',
                             'BLOCKED_SHOT',
                             'TAKEAWAY',
                             'GIVEAWAY',
                             'HIT',
                             'GOAL',
                             'PERIOD_START',
                             'PERIOD_END',
                             'EARLY_INT_START',
                             'PENALTY',
                             'STOP',
                             'SHOOTOUT_COMPLETE',
                             'GAME_END',
                             'EARLY_INT_END',
                             'EARLY_INT_END'])]
# rawDataFiltered = rawDataFiltered[rawDataFiltered['playerType'].isin(['NULL', 
#                                    'Winner', 
#                                    'Loser', 
#                                    'Hitter', 
#                                    'PlayerID','Shooter','Blocker','Unknown','Scorer','PenaltyOn','DrewBy'])]
# Sorting
rawDataFiltered = rawDataFiltered.sort_values(by=['gameID', 'periodNum','periodTime'])

In [28]:
def team_type(teamID, homeTeamID, awayTeamID):
    if teamID == homeTeamID:
        return 'HOME'
    elif teamID == awayTeamID:
        return 'AWAY'
    else:
        return np.nan

def zone(xCoord, teamType, periodNum):
    if teamType == np.nan:
        return 'NEUTRAL'
    if (xCoord >= -25) & (xCoord < 25):
        return 'NEUTRAL'
    else:
        if int(periodNum)%2 == 1:
            if (xCoord > 25) & (teamType == 'AWAY'):
                return 'OFFENSIVE'
            elif (xCoord < -25) & (teamType == 'AWAY'):
                return 'DEFENSIVE'
            elif (xCoord > 25) & (teamType == 'HOME'):
                return 'DEFENSIVE'
            elif (xCoord < -25) & (teamType == 'HOME'):
                return 'OFFENSIVE'
        else:
            if (xCoord > 25) & (teamType == 'AWAY'):
                return 'DEFENSIVE'
            elif (xCoord < -25) & (teamType == 'AWAY'):
                return 'OFFENSIVE'
            elif (xCoord > 25) & (teamType == 'HOME'):
                return 'OFFENSIVE'
            elif (xCoord < -25) & (teamType == 'HOME'):
                return 'DEFENSIVE'
            
def elapsed_seconds(periodNum, periodTime):
    return (int(periodNum) - 1) * 20 * 60 + periodTime.total_seconds()/60

class Queue:
    #CITATION: https://runestone.academy/runestone/books/published/pythonds/BasicDS/ImplementingaQueueinPython.html
    def __init__(self):
        self.queue = []

    def isEmpty(self):
        return self.queue == []

    def enqueue(self, item):
        self.queue.insert(0,item)

    def dequeue(self):
        return self.queue.pop()

    def size(self):
        return len(self.queue)
    
    def get_queue(self):
        return self.queue

    def exchange(self, oldItem, newItem):
        self.queue[self.queue.index(oldItem)] = newItem
        
    def remove(self, item):
        self.queue.remove(item)
        
    


In [29]:
rawDataFiltered['zone'] = rawDataFiltered.apply(lambda row: zone(row['xCoordinate'],
                                       team_type(row['teamID'], 
                                                 row['homeTeamID'], 
                                                 row['awayTeamID']), row['periodNum']) ,axis=1)

In [74]:
sequences = []
actionEvents = ['FACEOFF',
                'SHOT',
                'MISSED_SHOT',
                'BLOCKED_SHOT',
                'TAKEAWAY',
                'GIVEAWAY',
                'HIT',
                'GOAL']
startEndEvents = ['PERIOD_START',
                  'PERIOD_END',
                  'EARLY_INT_START',
                  'PENALTY',
                  'STOP',
                  'SHOOTOUT_COMPLETE',
                  'GAME_END',
                  'EARLY_INT_END',
                  'EARLY_INT_END']

sequenceNum = 0
eventNum = 0
context = {'goalDiff':0, 'manpowerDiff':0, 'periodNum':1}
penaltyQueue = Queue()
for index, row in rawDataFiltered.iterrows():
    # Computing if respective team is home or away
    teamType = team_type(row['teamID'], row['homeTeamID'], row['awayTeamID'])
        
    # Catching penalties to update the context
    if row['eventTypeID'] == 'PENALTY':
        if int(row['penaltyMinutes']) != 10:
            # Getting the end time of the penalty (in seconds)
            penaltyStart = elapsed_seconds(row['periodNum'],row['periodTime'])
            penaltyEnd = penaltyStart + row['penaltyMinutes'] * 60
            

            # Determining who took the penalty to update the context
            if teamType == 'HOME':
                context['manpowerDiff'] += 1
            else:
                context['manpowerDiff'] += -1

            # Enqueuing the penalty
            penaltyQueue.enqueue({'team':teamType, 
                                  'penaltyStart': penaltyStart, 
                                  'penaltyEnd':penaltyEnd, 
                                  'penaltyLength': 
                                  row['penaltyMinutes']})
#             print(penaltyQueue.get_queue())
#             print("***************************************************")
    else:
        # If there currently is a penalty in the penalty queue
        if penaltyQueue.size() > 0:
            # Determining the time of the action
            actionTime = elapsed_seconds(row['periodNum'],row['periodTime'])
            
            # Iterating through all  the penalties in the queue from oldest to newest
            for penalty in reversed(penaltyQueue.get_queue()):
                # If the action occured after the penalty ended, we update the context and pop the penalty
                if penalty['penaltyEnd'] < actionTime:
                    # Popping the penalty
                    penaltyQueue.remove(penalty)

                    # Updating the context
                    if penalty['team'] == 'HOME':
                        context['manpowerDiff'] -= 1
                    else:
                        context['manpowerDiff'] -= -1
            
        # If the event type is a goal
        if row['eventTypeID'] == 'GOAL':
            
            # Injecting the shot before the goal# Adding in the next sequency
            sequences.append([row['gameID'],  
                      context['goalDiff'],
                      context['manpowerDiff'],
                      context['periodNum'],
                      sequenceNum, 
                      eventNum, 
                      'SHOT',  # action type
                      teamType,  # Home/Away
                      row['zone']]) #Neutral/Offensive/Defensive/NULL
            # Incrementing the event number
            eventNum += 1
            
            # Updating the context
            if teamType == 'HOME':
                context['goalDiff'] -= 1
            else:
                context['goalDiff'] -= -1

            # Defining home/away flags to only pop off the minimum number of penalties
            FLAGS = {'HOME':True, 'AWAY':True}
            
            # Determining the time of the action
            actionTime = elapsed_seconds(row['periodNum'],row['periodTime'])
            
            # If there currently is a penalty in the penalty queue
            if penaltyQueue.size() > 0:
                
                # Iterating through all  the penalties in the queue from oldest to newest
                for penalty in reversed(penaltyQueue.get_queue()):
                    
                    # If the penalty is a 5 minute major, the player must serve the full 5 minutes (no change needed)
                    # If the penalty is over it would have been popped in the above if statement
                    if penalty['penaltyLength'] == 5:
                        continue
                    else:
                        
                        # Making sure its not a shorthanded goal and we haven't already popped a penalty for this goal/team
                        if (penalty['team'] != teamType) & (FLAGS[penalty['team']]):
                            
                            # Creating the updated penalty
                            newPenalty = penalty
                            newPenalty['penaltyStart'] = actionTime
                            newPenalty['penaltyLength'] += -120
                            newPenalty['penaltyEnd'] = newPenalty['penaltyStart'] + newPenalty['penaltyLength']
        
                            if penalty['penaltyLength'] <= 0:
                
                                # Popping the penalty off
                                penaltyQueue.remove(penalty)
                        
                                # Updating the context
                                if penalty['team'] == 'HOME':
                                    context['manpowerDiff'] -= 1
                                else:
                                    context['manpowerDiff'] -= -1
                            else:
                                
                                # replacing the old penalty info with the new one
                                penaltyQueue.exchange(penalty, newPenalty)
                                
                            FLAGS[penalty['team']] == False 
                            
    # Updating the context if needed
    if row['periodNum'] != context['periodNum']:
        context['periodNum'] = row['periodNum']
    
    # Adding in the next sequency
    sequences.append([row['gameID'],  
                      context['goalDiff'],
                      context['manpowerDiff'],
                      context['periodNum'],
                      sequenceNum, 
                      eventNum, 
                      row['eventTypeID'],  # action type
                      teamType,  # Home/Away
                      row['zone']]) #Neutral/Offensive/Defensive/NULL
    if row['eventTypeID'] in actionEvents:
        eventNum += 1
    else:
        sequenceNum += 1
        eventNum = 0
#     break
sequenceData = pd.DataFrame(sequences, 
                            columns=['gameID', 
                                     'goalDiff', 
                                     'manpowerDiff',
                                     'periodNum', 
                                     'sequenceNum',
                                     'eventNum',
                                     'event', 
                                     'team', 
                                     'zone'])

In [75]:
sequenceData.to_csv('delete.csv')

In [71]:
def R(action, team):
    print(action)
    print(team)
    if (action == 'GOAL') & (team == 'HOME'):
        return 1
    else:
        return 0

class Node():
    def __init__(self, context, action):
        self.context = context
        self.action = action
        self.children = []
        self.numOccurences = 0
        self.converged = False
        self.Q = 0
        self.currentValue = 0
        self.lastValue = 0
    
    def get_context(self):
        return self.context
    
    def get_action(self):
        return self.action
    
    def get_event(self):
        return self.event
        
    def update_count(self):
        self.numOccurences += 1
        
    def get_count(self):
        return self.numOccurences
        
    def insert_action(self, node, contexts, actions):
        # If we have reached the end of the current sequence
        if len(contexts) == 0:
            return
        # Iterates the nodes current children
        for child in node.children:

            # If this node already has an instance of this context/action
            if (child.get_context() == contexts[0]) & (child.get_action() == contexts[0]):

                # Update the node count
                child.update_count()
                
                # Popping this context off the list
                contexts.pop(0)
                
                # Popping this action off the list
                actions.pop(0)

                # Continue inserting the actions in the sequence
                insert_action(child, contexts, actions)
                return 0
        
        # If none of the children were a match or if the node has no children

        # Creating the new node
        newNode = Node(contexts[0], actions[0])

        # Incrementing its count
        newNode.update_count()

        # Adding to the list
        node.children.append(newNode)
        
        
        # Popping this context off the list
        contexts.pop(0)
        
        # Popping this action off the list
        actions.pop(0)
        
        # Continue inserting the actions in the sequence
        node.insert_action(newNode, contexts, actions)
        
    def traversal(self, node):
        print("**********************************************************************")
        print(f"Context: {node.get_context()}")
        print(f"Action: {node.get_action()}")
        print(f"Count: {node.get_count()}")
        for child in node.children:
            node.traversal(child)
            
    def mgm_traversal(self, node):
        for child in node.children:
            node.mgm_traversal(child)
        if ~node.converged:
            summation = 0
            for child in node.children:
                summation += child.numOccurences *  child.Q
            node.Q = R(node.action['event'], node.action['team']) + summation/node.numOccurences
            node.currentValue += abs(node.Q)    
        
class Tree():
    def __init__(self):
        self.root = Node({'goalDiff':None, 
                              'manpowerDiff':None, 
                              'period':None},
                             {'event':None,
                              'team':None,
                              'zone':None})
        self.currentNode = self.root
        
    def create_tree(self, contexts, actions):
        self.root.update_count()
        
        # Iterating through the nodes current children
        for child in self.root.children:
            
            # If this node already has an instance of this context
            if child.get_context() == contexts[0]:
                # Increment the count
                child.update_count()
                
                # Begin recursion
                child.insert_action(child, contexts, actions)
                return   
        
        # If none of the children were a match or if the node has no children
        
        # Creating the new node (note this is purely a context node, so it has no action)
        newNode = Node(contexts[0], 
                       {'event':None,
                        'team':None,
                        'zone':None})
        
        # Incrementing its count
        newNode.update_count()
        
        # Adding to the list
        self.root.children.append(newNode)
        
        # Begin recursion
        self.root.insert_action(newNode, contexts, actions)
        
    def tree_traversal(self):
        print("ROOT")
        print(f"Context: {self.root.get_context()}")
        print(f"Action: {self.root.get_action()}")
        print(f"Count: {self.root.get_count()}")
        for child in self.root.children:
            self.root.traversal(child)
            
    def markov_game_model(self, M = 100000, c = 0.0001):
        for i in range(1,M):
            self.root.mgm_traversal(self.root)
            
        
        


In [72]:
tree = Tree()
for sequence in sequenceData['sequenceNum'].unique():
    subSequence = sequenceData[sequenceData['sequenceNum'] == sequence]
    contexts = subSequence[['goalDiff','manpowerDiff','periodNum']].to_dict('records')
    actions = subSequence[['event','team','zone']].to_dict('records')
    tree.create_tree(contexts=contexts, actions=actions)

In [73]:
tree.tree_traversal()

ROOT
Context: {'goalDiff': None, 'manpowerDiff': None, 'period': None}
Action: {'event': None, 'team': None, 'zone': None}
Count: 60
**********************************************************************
Context: {'goalDiff': 0, 'manpowerDiff': 0, 'periodNum': '1'}
Action: {'event': None, 'team': None, 'zone': None}
Count: 7
**********************************************************************
Context: {'goalDiff': 0, 'manpowerDiff': 0, 'periodNum': '1'}
Action: {'event': 'PERIOD_START', 'team': nan, 'zone': None}
Count: 1
**********************************************************************
Context: {'goalDiff': 0, 'manpowerDiff': 0, 'periodNum': '1'}
Action: {'event': 'FACEOFF', 'team': 'AWAY', 'zone': 'NEUTRAL'}
Count: 1
**********************************************************************
Context: {'goalDiff': 0, 'manpowerDiff': 0, 'periodNum': '1'}
Action: {'event': 'STOP', 'team': nan, 'zone': None}
Count: 1
*********************************************************************

In [67]:
tree.markov_game_model(M=10)

PERIOD_START
nan
STOP
nan
FACEOFF
AWAY
STOP
nan
HIT
AWAY
BLOCKED_SHOT
HOME
FACEOFF
AWAY
STOP
nan
MISSED_SHOT
AWAY
FACEOFF
AWAY
STOP
nan
SHOT
AWAY
SHOT
AWAY
SHOT
AWAY
BLOCKED_SHOT
HOME
HIT
HOME
GIVEAWAY
HOME
MISSED_SHOT
HOME
BLOCKED_SHOT
AWAY
HIT
HOME
HIT
HOME
HIT
AWAY
SHOT
HOME
TAKEAWAY
HOME
FACEOFF
AWAY
STOP
nan
BLOCKED_SHOT
AWAY
GIVEAWAY
AWAY
MISSED_SHOT
AWAY
TAKEAWAY
AWAY
HIT
HOME
HIT
AWAY
BLOCKED_SHOT
AWAY
HIT
AWAY
FACEOFF
AWAY
STOP
nan
HIT
AWAY
TAKEAWAY
HOME
GOAL
AWAY
SHOT
AWAY
FACEOFF
HOME
BLOCKED_SHOT
HOME
HIT
AWAY
SHOT
AWAY
SHOT
AWAY
HIT
AWAY
FACEOFF
HOME


TypeError: 'NoneType' object is not subscriptable